*Music Enabled Running - FR Corp*

# **Exploratory Data Analysis - Test Person Two**

In this notebook you can see the exploratory data analysis made from the data of the second testperson from the project Music Enabled Running. The notebook consists of multiple chapters, where data will be imported, converted, filtered and analysed.

In [1]:
#Necessary to load in files from Google Drive with Colaboratory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1.Libraries**

In [2]:
import pandas as pd        
import numpy as np

import seaborn as sns
import matplotlib as mat
import matplotlib.pyplot as plt   
%matplotlib inline

import time
from datetime import datetime

import scipy as sy
import scipy.fftpack as syfp
import pylab as pyl

import plotly.express as px

## **2.Dataset Import**

In [3]:
footpods = pd.read_csv('/content/drive/MyDrive/ICT Fontys/Groupprojects/Semester 7/Mini Company - FR Corp/1.Projects/Music/2.Exploratory Data Analysis/Datasets/TestRunnerTwo/footpods.csv')
sessions = pd.read_csv('/content/drive/MyDrive/ICT Fontys/Groupprojects/Semester 7/Mini Company - FR Corp/1.Projects/Music/2.Exploratory Data Analysis/Datasets/TestRunnerTwo/sessions.csv')
footpods_sc = pd.read_csv('/content/drive/MyDrive/ICT Fontys/Groupprojects/Semester 7/Mini Company - FR Corp/1.Projects/Music/2.Exploratory Data Analysis/Datasets/TestRunnerTwo/footpods_sc.csv')
music = pd.read_csv('/content/drive/MyDrive/ICT Fontys/Groupprojects/Semester 7/Mini Company - FR Corp/1.Projects/Music/2.Exploratory Data Analysis/Datasets/TestRunnerTwo/music.csv')

FileNotFoundError: ignored

## **3.Data Understanding**

In this chapter, we need to understand what the data is about in order to do the data converting and filtering. 

In [ ]:
#footpods dataset
footpods.info()

In [ ]:
#footpods_sc dataset
footpods_sc.info()

In [ ]:
#session dataset
sessions.info()

In [ ]:
#music dataset
music.info()

### **3.1.Basic Metrics**

In [ ]:
#getting to know the dataset of footpods
footpods.describe()

In [ ]:
#count of the steps in footpods
count_foot = footpods.groupby('foot')['power'].count().reset_index()
count_foot

In [ ]:
#getting to know the dataset of footpods_sc
footpods_sc.describe()

In [ ]:
#count of the steps in footpods_sc
count_foot_sc = footpods_sc.groupby('foot')['t'].count().reset_index()
count_foot_sc

In [ ]:
#getting to know the dataset of sessions
sessions.describe()

In [ ]:
#getting to know the dataset of music
music.describe()

In [ ]:
#see if there are any null values in footpods dataset
footpods.isnull().sum()

In [ ]:
#see if there are any null values in footpods_sc dataset
footpods_sc.isnull().sum()

In [ ]:
#see if there are any null values in sessions dataset
sessions.isnull().sum()

In [ ]:
#see if there are any null values in music dataset
music.isnull().sum()

You can see above, that in the `music` dataset, there are 679 null values in context. 

### **3.2.Variables and Data Types**

In [ ]:
footpods.dtypes

In [ ]:
sessions.dtypes

In [ ]:
footpods.shape

In [ ]:
sessions.shape

## **4.Data Converting**
In this chapter we are going to convert the data in the correct way. 

### **4.1.Data Type Conversion**

In [ ]:
sessions.info()

In [ ]:
#footpods data conversion
footpods['t'] = pd.to_datetime(footpods['t'])

#footpods_sc data conversion
footpods_sc['t'] = pd.to_datetime(footpods_sc['t'])

#sessions data conversion
sessions['t_start'] = pd.to_datetime(sessions['t_start'])
sessions['t_end'] = pd.to_datetime(sessions['t_end'])

#music data conversion 
music['t'] = pd.to_datetime(music['t'])

 ### **4.2.Split Date and Time Data**

In [ ]:
#split the date and time
footpods['date'] = [d.date() for d in footpods['t']]
footpods['time'] = [d.time() for d in footpods['t']]

In [ ]:
#convert it to the correct datatype
footpods['date'] = pd.to_datetime(footpods['date'])
footpods['time'] = pd.to_datetime(footpods['t'], format = '%H:%M:%S.%f').dt.time[0]

In [ ]:
#rename the sessions columns name
sessions = sessions.rename(columns={'session_id': 'session_id', 'user_id': 'user_id', 't_start': 't', 't_end': 't_end', 'duration': 'duration'})

In [ ]:
# #split the date and time for sessions
sessions['date'] = [d.date() for d in sessions['t']]
sessions['time'] = [d.time() for d in sessions['t']]

In [ ]:
#convert it to the correct datatype for sessions
sessions['date'] = pd.to_datetime(sessions['date'])
sessions['time'] = pd.to_datetime(sessions['t'], format = '%H:%M:%S.%f').dt.time[0]

### **4.3.Frequency Calculation**

In [ ]:
#Time difference based on both feet.
footpods['timediff'] = footpods['t'].diff()
#Convert the time difference in seconds.
footpods['timediff'] = footpods['timediff'].dt.total_seconds()
#Calculate the frequency based on the time difference.
footpods['frequency'] = 1 / footpods['timediff']
#Remove first row with null timediff value.
footpods = footpods.iloc[1: , :]

In [ ]:
footpods['frequency'].head(10)

Now that we have our data filtering on the `footpods` and `sessions`, we are going to merge these 

## **5.Data Filtering on Music**

In this chapter we are going to filter the "correct" sessions based on the amount of songs that was played during the sessions.

### **5.1.Merge**

Now we are going to merge the 4 dataset. `footpods`, `footpods_sc`, `sessions` & `music`. First we are going to merge the `footpods` and `footpods_sc` together. Then we are going to merge the `sessions` and then the `music` dataset. 


#### **5.1.1 Merge footpods & footpod.sc**


In [ ]:
#check the amount of rows of footpods
footpods.shape

In [ ]:
#check the amount of rows of footpod_sc
footpods_sc.shape

In [ ]:
#merge the two dataset based on the nearest time
df_combined = pd.merge_asof(footpods, footpods_sc, on='t', direction = 'nearest')

#### **5.1.2 Merge combined & sessions**


In [ ]:
#check the amount of rows of the new combined dataset
df_combined.shape

In [ ]:
#check the amount of rows of sessions
sessions.shape

In [ ]:
#merge the two dataset based on the nearest time
df_combined1 = pd.merge_asof(df_combined, sessions, on='t', direction = 'nearest')

In [ ]:
#check the amount of rows of the new dataset
df_combined1.shape

You can see that the amount of rows are the same, which means that it is correct because we only wanted to "label" our `df_combined` dataset with the correct `session_id`. 

#### **5.1.3 Merge combined & music**

In [ ]:
#check the amount of rows of music
music.shape

In [ ]:
#merge the two dataset based on backward
df_combined2 = pd.merge_asof(df_combined1, music, on='t', direction = 'backward')

In [ ]:
#check the amount of rows on the new final dataset
df_combined2.shape

And over here as wel, you can see that the amount of rows are the same which means that it is correct because we want to see which songs were played. 

In [ ]:
#name the correct dataframe
df = df_combined2

In [ ]:
#see for the final result
df.head()

### **5.1.Count Music per Session**

Now we are going to filter out the so-called bad sessions. We label a session as bad when the duration of the session is less than 30 minutes because test sessions are also in this dataset. So, to filter the bad sessions out, we are going to count the amount of music that was played within each session. This way we can see how many songs were played and we can "easily" determine if a session is valuable or not. 

In [ ]:
#To have a better overview, merge the music and sessions dataset
df_music_sessions= pd.merge_asof(music, sessions, on='t', direction = 'backward')

#grouping on date assuming we have only one session per day, and count the unique amount of songs that was played during that day 
df_date_music = df_music_sessions.groupby('date')['track_uri'].nunique().reset_index()

#filter based on per session_id
df_sessionid_music = df_music_sessions.groupby('session_id')['track_uri'].nunique().reset_index()

In [ ]:
#visualisation of amount of song played per date
fig = px.bar(df_date_music, x="date", y="track_uri", 
             labels={
                     "date": "Date",
                     "track_uri": "Amount of song played",
                 },
                title="Count of Songs Played per Day")
fig.show()

In [ ]:
#visualisation of amount of song played per date
fig = px.bar(df_sessionid_music, x="session_id", y="track_uri", 
             labels={
                     "session_id": "Session ID",
                     "track_uri": "Amount of song played",
                 },
                title="Count of Songs Played per Session")
fig.show()

#### **INFO: Average lenght of song** 
According to a report of Quarz, the average song on 2018 on the Billboard Hot 100 is about 3 minutes and 30 seconds long. 

https://qz.com/1519823/is-spotify-making-songs-shorter/


So to determine how long a good session is, we need to do some calculations. 

In [ ]:
#calculate the duration of music per day
df_date_music['duration w music'] = df_date_music['track_uri'] * 3.5
df_date_music.head()

In [ ]:
#calculate the duration of music per session
df_sessionid_music['duration w music'] = df_sessionid_music['track_uri'] * 3.5
df_sessionid_music

You can see here that there are a lot of sessions where only 1 song was played. The explanation for this is that the test person was probably testing the equipments out. We are only interested in sessions where he ran. 

So in this case, we are going to filter out the sessions where less than 4 songs were played. We are only interested in sessions where the music duration is **atleast 20-25 minutes** and **maximum of 75 minutes**. 

In [ ]:
#filter this with the dataset grouped on day
df_date_music_good = df_date_music[(df_date_music['duration w music'] < 75) & (df_date_music['duration w music'] > 20)]

df_date_music_good

In [ ]:
#filter this with the dataset grouped on session id
df_sessionid_music_good = df_sessionid_music[(df_sessionid_music['duration w music'] < 75) & (df_sessionid_music['duration w music'] > 20)]
# df_sessionid_music_test = df_sessionid_music[(df_sessionid_music['track_uri'] < 20) & (df_sessionid_music['track_uri'] > 4 )]

df_sessionid_music_good

So as you can see, we are left with not so many sessions since a lot of sessions were bad sessions like we mentioned before. 

The downside of grouping on date is that it's possible that multiple sessions took place on one day and is therefore not the most safest way to filter out based on date. Therefore, we are going to filter the sessions out based on sessionid. This way we are sure we are only selecting the right session. 

### **5.2.Drop Sessions**

In this section, we are going to drop the bad sessions.

In [ ]:
#filtering the right sessions 
df = df[(df['session_id'] == '15FB2F71-7F56-4434-8A3B-127A14E966B2') |
                      (df['session_id'] == '321E5D99-0074-4ED5-A2D2-74DDF96FD394') |
                      (df['session_id'] == '3CCAC32B-C713-469E-978C-0A4452E04BC6') |
                      (df['session_id'] == '6C43D7B8-3D59-4230-9CE0-2ECA315A3456') |
                      (df['session_id'] == '848F78F2-3AAD-4A8B-81EC-2D3C6E13FB0B') |
                      (df['session_id'] == '8E684098-83E7-4038-B15A-B5A69DD902DD')].reset_index()

df

### **5.3.Removing Outliers**

In this chapter we are going to focus on the outliers and drop these. What we are interested in is the running dataset.

We are going to focus on the time difference and only on the low values. The lower the value, the faster this person was taking his/her steps therefore he or she was running.

In [ ]:
#calculating statistical data 
df[['timediff']].describe()

In [ ]:
#look for outliers with the help of the boxplot
fig = px.box(df[['timediff']], y="timediff")
fig.show()

In [ ]:
# Dropping the lower and upper fance values that fall out the boxplot
df = df.drop(df[(df['timediff'] < 0.0358)].index)
df = df.drop(df[(df['timediff'] > 0.8053)].index)

In [ ]:
#check the dataset
df

In [ ]:
#check the boxplot again now that the outliers are removed
plt = px.box(df, y = 'timediff')
plt.show()

In [ ]:
#calculating statistical data 
df.describe()

You can see that the column `flight_ratio` has minus values. We are going to drop all the values below 0 for this column. 

In [ ]:
#drop all the values from flight_ratio that is lower than 0
df = df.drop(df[(df['flight_ratio'] <= 0)].index)

In [ ]:
#calculating statistical data 
df.describe()

You can see above, that for frequency we still have some outliers. With the help of boxplot as we did before, we are going to remove these outliers. 

In [ ]:
#look for outliers with the help of the boxplot
fig = px.box(df[['frequency']], y="frequency")
fig.show()

In [ ]:
df = df.drop(df[(df['frequency'] > 5.006259)].index)

In [ ]:
#check the boxplot again now that the outliers are removed
plt = px.box(df, y = 'frequency')
plt.show()

Now that we have cleared most of our dataset, we want to select only the useful columns.

In [ ]:
#drop the columns
df = df.drop(["index", "foot_y", "user_id", 'date_y', 'time_y', 'repeat_mode', "shuffle", "crossfade", "t_end"], axis=1)

#change the name of the df
df = df.rename(columns={'foot_x': 'foot', 'date_x': 'date', 'time_x': 'time'})

## **6.Analysis**

In this chapter we are going to analyse our cleaned dataset to see if there is any correlation or anything remarkable going on. 

### **6.1.Basic Correlations**
For the basic correlation, we are going to focus on one session and preferably the longest. 

In [ ]:
#find the longest session
df_longest = df.groupby("session_id")['duration'].max().reset_index()

#session 15FB2F71-7F56-4434-8A3B-127A14E966B2	is the longest

df_longest = df[(df['session_id'] == '15FB2F71-7F56-4434-8A3B-127A14E966B2')].reset_index()

In [ ]:
fig = px.scatter_matrix(df_longest,
    dimensions=["pronation", "braking", "contact_time", "flight_ratio","power", "timediff","frequency"],
    color="foot", symbol="foot",
    title="Pairplot of Test Person #2",
    labels={col:col.replace('_', ' ') for col in df_longest.columns})
fig.update_layout(
    autosize=False,
    width=900,
    height=800,
    font=dict(size=8))
fig.update_traces(
    marker={'size':2},
    diagonal_visible=False)
fig.show()

In [ ]:
df_cor = df.drop(df['duration'].index)
corr = df.corr()
corr.style.background_gradient(cmap = 'coolwarm')

### **6.2.Amount of Steps**

In [ ]:
df_footsteps = df_longest.groupby('foot')['t'].count().reset_index()
df_footsteps.columns = ["foot", "count"]

fig = px.bar(df_footsteps, x="foot", y="count", color = 'foot', 
             labels = {
                     "foot": "Foot",
                     "count": "Amount of Steps",
                 },
                title = "The Amount of Steps by each Foot")
fig.show()

### **6.3.Frequency**

In this paragraph we want to see how many steps TestPerson2 took in one second. 

In [ ]:
fig = px.line(df_longest, x="t", y="frequency", title='Frequency during Longest Session')
fig.show()

### **6.4. Contact Time and Flight Ratio**

In [ ]:
fig = px.scatter(df, x = "flight_ratio", y = "contact_time", color = "foot",
                 labels = {
                     "flight_ratio": "Flight Ratio in miliseconds",
                     "contact_time": "Contact Time in miliseconds",
                     "foot": "Foot of Runner"
                 },
                title = "Correlation between Contact Time and Flight Ratio")
fig.show()

As you can see, the left foot is plotting over the right foot. Therefore we are going to use a 3D model to have a better view. 

In [ ]:
df = df_longest
fig = px.scatter_3d(df_longest, 
                    x = 'flight_ratio', 
                    y = 'contact_time', 
                    z = 'foot',
                    color = 'foot')
fig.show()

## **7.Save CSV-file**

Save the cleaned version in a CSV file 

In [ ]:
df.columns

In [ ]:
# df.to_csv('testperson2.csv', index=False)
df.to_csv(r'\content\drive\MyDrive\ICT Fontys\Groupprojects\Semester 7\Mini Company - FR Corp\1.Projects\Music\2.Exploratory Data Analysis\Datasets\TestRunnerTwo\testperson2.csv', index=False)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')